In [103]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from absl import logging
logging.set_verbosity(logging.INFO)
from open_spiel.python.examples.ubc_nfsp_example import NFSPPolicies, policy_from_checkpoint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [104]:
### Load game
game_name = 'big_game_2'
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{game_name}.json'))


In [105]:
# game.new_initial_state().information_state_tensor(0)

In [111]:
policy = policy_from_checkpoint('/shared/outputs/dec7_big_game_2/gru_deep', 'checkpoint_latest').nfsp_policies

INFO:absl:Game loaded
INFO:absl:Using game instance: turn_based_simultaneous_game
INFO:absl:Game has a state size of 6010, 12 distinct actions, and 2 players
INFO:absl:Game has 3 products


In [112]:
# solver = pyspiel.CFRSolver(game)
# for i in range(1000):
#     solver.evaluate_and_update_policy()
# policy = solver.average_policy()

In [113]:
def make_policy_decorators(policy):
    def edge_weight_by_policy_decorator(parent_state, unused_child, action, **kwargs):
        attrs = ubc_treeviz.default_edge_decorator(parent_state, unused_child, action)  # get default attributes
        if parent_state.current_player() < 0:
            return attrs
        
        if '@ $0' in attrs['label']:
            attrs['label'] = 'Drop Out'
        
        action_prob = policy.action_probabilities(parent_state)[action]
        attrs['arrowsize'] = action_prob * attrs['arrowsize']
        attrs['penwidth'] = action_prob
        attrs['label'] = f'[{action_prob:.2f}] {attrs["label"]}'
#         print(attrs, parent_state.current_player())
        return attrs

    def node_weight_by_policy_decorator(state, **kwargs):
        attrs = ubc_treeviz.default_node_decorator(state)
        if state.is_terminal():
            return_list = eval(attrs['label'])
            attrs['label'] = ', '.join([f'{x:.2f}' for x in return_list])
        
        # TODO: If terminal, report allocation
        # {'label': 'Current player: 0\np0v125, 125b150\n', 'fontsize': 8, 'width': 0.25, 'height': 0.25, 'margin': 0.01, 'shape': 'square', 'color': 'blue'}
        #         print(attrs)
        return attrs

    return node_weight_by_policy_decorator, edge_weight_by_policy_decorator

node_policy_decorator, edge_policy_decorator = make_policy_decorators(policy)

#   player = parent.current_player()
#   attrs = {
#       "label": " " + parent.action_to_string(player, action),
#       "fontsize": _FONTSIZE,
#       "arrowsize": _ARROWSIZE
#   }
#   attrs["color"] = _PLAYER_COLORS.get(player, "black")
#   return attrs


In [114]:
gametree = ubc_treeviz.GameTree(
    game,
    node_decorator=node_policy_decorator,
    edge_decorator=edge_policy_decorator,
    group_infosets=True,
    group_terminal=False,
    group_pubsets=False, 
    target_pubset='*',
    depth_limit=5,
#     state_prob_limit=0.1,
    action_prob_limit=0.1,
    policy=policy
)

# outfile = f'./{game_name}.pdf'
outfile= f'./{game_name}_rnn.pdf'

# logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(outfile, prog='dot')
logging.info("Game tree saved to file: %s", outfile)


INFO:absl:Building tree...
INFO:absl:Built tree!
INFO:absl:Game tree saved to file: ./big_game_2_rnn.pdf


In [ ]:
# TODO: Given a STRATEGY, colour the lines according to thickness?